In [0]:
--Multi-Layer Modeling Trading Signals

with base as (
  select
    ticker,
    trade_date,
    close,
    volume,
    lag(close) over (
      partition by ticker
      order by trade_date
    ) as prev_close
  from analytics.stock_prices
),

returns as (
  select 
    ticker,
    trade_date,
    close,
    volume,
    (close - prev_close) / prev_close as daily_return
  from base
  where prev_close is not null
),

numbered as (
  select *,
  row_number() over (
    partition by ticker
    order by trade_date
  ) as rn
 from returns
),

indicator as (
  select 
    ticker,
    trade_date,
    close,
    volume,

    --20 Day Moving Average (strict)

    case when rn >= 20 then 
      avg(close) over (
        partition by ticker
        order by trade_date
        rows between 19 preceding and current row
      )
    end as ma_20,

    --Distance from Moving Average

    case when rn >= 20 then 
      (close - avg(close) over (
        partition by ticker
        order by trade_date
        rows between 19 preceding and current row
      ))
      / 
      avg(close) over (
        partition by ticker
        order by trade_date
        rows between 19 preceding and current row
      )
    end as pct_from_ma20,

   --30 Day Rolling Volatility (strict)

   case when rn >= 30 then 
    stddev_samp(daily_return) over (
      partition by ticker
      order by trade_date
      rows between 29 preceding and current row
    )
   end as vol_30,

  --Rolling Max for Drawdown
  max(close) over (
    partition by ticker
    order by trade_date
    rows between unbounded preceding and current row
  ) as rolling_max,

  --20 Day Volume Average

  case when rn >= 20 then 
    avg(volume) over (
      partition by ticker
      order by trade_date
      rows between 19 preceding and current row
    )
  end as avg_vol_20
 
  from numbered
),

final as (
  select 
    ticker,
    trade_date,
    close,
    volume,
    ma_20,
    round(pct_from_ma20 * 100, 2) as pct_from_m20,
    round(vol_30 *100, 2) as vol_30_pct,
    round((close - rolling_max) / rolling_max * 100, 2) as drawdown,
    avg_vol_20,
    volume / avg_vol_20 as vol_ratio
  from indicator
)

--Filter to latest trading date
select *
from final
where trade_date = (
  select max(trade_date) 
  from analytics.stock_prices
  )
and ma_20 is not null
and vol_30_pct is not null
and pct_from_m20 > 0
and drawdown > -5
and vol_ratio > 1.5
order by vol_ratio desc;

--Filter to specific trading date
select *
from final
where trade_date = '2026-02-19'
and ma_20 is not null
and vol_30_pct is not null
and pct_from_m20 > 0
and drawdown > -5
and vol_ratio > 1.5
order by vol_ratio desc;